In [72]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import BitsAndBytesConfig
from datasets import load_from_disk
import torch
import os
import torch

In [73]:
def load_model_and_tokenizer(model_name: str, local_dir: str = "./model/TinyLlama"):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    if os.path.exists(local_dir) and os.path.isdir(local_dir):
        print(f"🔁 Загрузка модели из локального каталога: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(local_dir)
        model = AutoModelForCausalLM.from_pretrained(
            local_dir,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=bnb_config
        )
    else:
        print(f"⬇️ Модель не найдена локально, загружаем из интернета и сохраняем в: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=bnb_config
        )
        # Сохраняем модель и токенизатор в локальный каталог
        tokenizer.save_pretrained(local_dir)
        model.save_pretrained(local_dir)

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    return tokenizer, model


In [74]:
def apply_lora(model):
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.gradient_checkpointing_enable()  

    return model

In [75]:
def tokenize_russian_dialogues(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue  # пропускаем пустые

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [ ]:
def tokenize_gsm8k_and_MAWPS(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [ ]:
def tokenize_asdiv(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["text"], batch["target"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [ ]:
def tokenize_mathqa(batch, tokenizer, max_length=512):
    texts = []

    for question, chain, result in zip(batch["question"], batch["chain"], batch["result"]):
        if question is None or chain is None or result is None:
            continue

        prompt = question.strip()
        response = f"{chain.strip()} → {result.strip()}"

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [76]:
def tokenize_dataset(dataset, tokenizer, tokenize_fn, max_length=512):
    def wrapped(example):
        return tokenize_fn(example, tokenizer, max_length=max_length)

    return dataset.map(wrapped, batched=True, remove_columns=dataset.column_names)

In [77]:
def train_lora(
    model_name: str,
    dataset_path: str,
    output_dir: str,
    epochs: int = 3,
    batch_size: int = 4,
    lr: float = 2e-4,
    max_length: int = 512,
    tokenize_fn=None
):
    # Загрузка
    tokenizer, model = load_model_and_tokenizer(model_name)
    model = apply_lora(model)
    dataset = load_from_disk(dataset_path)
    dataset = tokenize_dataset(
        dataset,
        tokenizer=tokenizer,
        tokenize_fn=tokenize_fn,
        max_length=max_length
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # обязательно для CausalLM
    )


    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.deterministic = False


    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=True,
        logging_steps=20,
        save_steps=200,
        save_total_limit=2,
        report_to="none"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator  
    )

    # Обучение
    trainer.train()
    model.save_pretrained(output_dir)

    del model
    del tokenizer
    torch.cuda.empty_cache()

In [78]:
def merge_lora_adapter(
    base_model_path: str,
    adapter_path: str,
    save_path: str
):
    # Загружаем базовую модель и токенизатор через универсальную функцию
    tokenizer, model = load_model_and_tokenizer(base_model_path)

    # Загружаем LoRA и объединяем
    model = PeftModel.from_pretrained(model, adapter_path)
    model = model.merge_and_unload()

    # Сохраняем объединённую модель
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"✅ Объединённая модель сохранена в: {save_path}")

    del model
    del tokenizer
    torch.cuda.empty_cache()

In [79]:
train_lora(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset_path="./data/russian_dialogues_20k",
    output_dir="./checkpoints/step_dialogue",
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_russian_dialogues
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                  # ← локально сохранённая/обновлённая модель
    adapter_path="./checkpoints/step_dialogue",           # ← путь к обученному адаптеру
    save_path="./model/TinyLlama"                         # ← перезаписываем модель
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 50000/50000 [00:12<00:00, 3941.04 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_27900\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,2.975200
40,2.583300
60,1.944100
80,1.839800
100,1.795600
120,1.789400
140,1.789300
160,1.789900
180,1.814500
200,1.787600


KeyboardInterrupt: 